In [5]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1010)>


False

In [6]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from tqdm import tqdm
import os
import glob

# Inicializa avaliadores
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smoothie = SmoothingFunction().method4

# Carrega a base completa
df_total = pd.read_csv('data/wikihow_results.csv')

# Define o tamanho do lote
batch_size = 200
num_batches = (len(df_total) + batch_size - 1) // batch_size  # Arredonda para cima

# Cria pasta de saída
output_dir = 'lotes_avaliados'
os.makedirs(output_dir, exist_ok=True)

# Lê todos os CSVs existentes e junta os títulos processados
csvs_existentes = glob.glob(os.path.join(output_dir, '*.csv'))
titulos_processados = set()
for path in csvs_existentes:
    try:
        df_existente = pd.read_csv(path, usecols=['title'])
        titulos_processados.update(df_existente['title'].dropna().unique())
    except Exception as e:
        print(f"⚠️ Erro ao ler '{path}': {e}")

# Processa lote a lote
for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(df_total))
    df_batch = df_total.iloc[start:end].copy()

    # Verifica se todos os títulos desse lote já estão nos processados
    titulos_lote = set(df_batch['title'].dropna().unique())
    if titulos_lote.issubset(titulos_processados):
        print(f"⏩ Lote {i+1} já processado com base nos títulos. Pulando...\n")
        continue

    print(f"🔄 Processando lote {i+1}/{num_batches}...")

    # TEXTOS
    textos = df_batch['text'].fillna('').astype(str).tolist()
    headlines = df_batch['headline'].fillna('').astype(str).tolist()
    resumos = df_batch['resumo'].fillna('').astype(str).tolist()

    # Métricas de HEADLINE vs TEXT
    rouge_h, bleu_h = [], []
    for ref, hyp in tqdm(zip(textos, headlines), total=len(textos), desc="→ ROUGE/BLEU HEADLINE"):
        scores = rouge.score(ref, hyp)
        rouge_h.append((scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure))

        ref_tokens = [nltk.word_tokenize(ref.lower())]
        hyp_tokens = nltk.word_tokenize(hyp.lower())
        bleu_h.append(sentence_bleu(ref_tokens, hyp_tokens, smoothing_function=smoothie))

    # BERTScore HEADLINE vs TEXT (em inglês)
    P_h, R_h, F1_h = bert_score(headlines, textos, lang="en", rescale_with_baseline=False)

    # BERTScore RESUMO vs TEXT (pt vs en)
    P_r, R_r, F1_r = bert_score(resumos, textos, lang="multilingual", rescale_with_baseline=False)

    # Adiciona ao DataFrame
    df_batch['headline_ROUGE-1'] = [r[0] for r in rouge_h]
    df_batch['headline_ROUGE-2'] = [r[1] for r in rouge_h]
    df_batch['headline_ROUGE-L'] = [r[2] for r in rouge_h]
    df_batch['headline_BLEU'] = bleu_h
    df_batch['headline_BERTScore_F1'] = F1_h.tolist()
    df_batch['resumo_BERTScore_F1'] = F1_r.tolist()

    # Comparação final
    df_batch['resumo_melhor_que_headline'] = df_batch['resumo_BERTScore_F1'] > df_batch['headline_BERTScore_F1']

    # Salva o lote
    output_path = os.path.join(output_dir, f'avaliacao_resumos_lote_{i}.csv')
    df_batch.to_csv(output_path, index=False)
    print(f"✅ Lote {i+1} salvo em '{output_path}'\n")

print("🏁 Todos os lotes foram processados com sucesso!")


⏩ Lote 1 já processado com base nos títulos. Pulando...

⏩ Lote 2 já processado com base nos títulos. Pulando...

⏩ Lote 3 já processado com base nos títulos. Pulando...

⏩ Lote 4 já processado com base nos títulos. Pulando...

⏩ Lote 5 já processado com base nos títulos. Pulando...

⏩ Lote 6 já processado com base nos títulos. Pulando...

⏩ Lote 7 já processado com base nos títulos. Pulando...

⏩ Lote 8 já processado com base nos títulos. Pulando...

⏩ Lote 9 já processado com base nos títulos. Pulando...

⏩ Lote 10 já processado com base nos títulos. Pulando...

⏩ Lote 11 já processado com base nos títulos. Pulando...

⏩ Lote 12 já processado com base nos títulos. Pulando...

⏩ Lote 13 já processado com base nos títulos. Pulando...

⏩ Lote 14 já processado com base nos títulos. Pulando...

⏩ Lote 15 já processado com base nos títulos. Pulando...

⏩ Lote 16 já processado com base nos títulos. Pulando...

⏩ Lote 17 já processado com base nos títulos. Pulando...

⏩ Lote 18 já processado

→ ROUGE/BLEU HEADLINE: 100%|██████████| 200/200 [00:00<00:00, 296.70it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Lote 335 salvo em 'lotes_avaliados/avaliacao_resumos_lote_334.csv'

🔄 Processando lote 336/386...


→ ROUGE/BLEU HEADLINE: 100%|██████████| 200/200 [00:00<00:00, 274.53it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Lote 336 salvo em 'lotes_avaliados/avaliacao_resumos_lote_335.csv'

🔄 Processando lote 337/386...


...

✅ Lote 384 salvo em 'lotes_avaliados/avaliacao_resumos_lote_383.csv'

🔄 Processando lote 385/386...


→ ROUGE/BLEU HEADLINE: 100%|██████████| 200/200 [00:01<00:00, 114.32it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Lote 385 salvo em 'lotes_avaliados/avaliacao_resumos_lote_384.csv'

🔄 Processando lote 386/386...


→ ROUGE/BLEU HEADLINE: 100%|██████████| 6/6 [00:00<00:00, 154.64it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Lote 386 salvo em 'lotes_avaliados/avaliacao_resumos_lote_385.csv'

🏁 Todos os lotes foram processados com sucesso!


In [9]:
import glob

all_files = glob.glob("lotes_avaliados/avaliacao_resumos_lote_*.csv")
df_final = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
df_final.to_csv("avaliacao_resumos_completa.csv", index=False)


In [10]:
df_final.head()

,headline,title,text,resumo,classificacao,metadata,max_length,language,nivel_ensino,status,erro,headline_ROUGE-1,headline_ROUGE-2,headline_ROUGE-L,headline_BLEU,headline_BERTScore_F1,resumo_BERTScore_F1,resumo_melhor_que_headline
0,"Understand what a prenuptial agreement does.,\...",How to Obtain a Prenuptial Agreement,"Put simply, a prenuptial agreement is a legal ...",NaN,NaN,NaN,500,pt-BR,medio,erro,"HTTP 422: {""detail"":[{""type"":""string_too_long""...",0.117717,0.067817,0.086801,3.690934e-07,0.840141,0.000000,False
1,"Calculate your budget for the wand.,\nChoose t...",How to Make a Wedding Ribbon Wand,How much money are you willing to spend on a w...,Este texto é um guia para fazer varinhas perso...,NaN,"{'tempo_processamento': 8.99620270729065, 'tam...",500,pt-BR,medio,ok,NaN,0.140187,0.049180,0.086449,2.049932e-06,0.814617,0.618793,False
2,Buy or make a neat hold-all to keep everything...,How to Make a Bride Survival Kit,Ideally it should be compact and easily carrie...,Este texto fala sobre montar um kit de emergên...,NaN,"{'tempo_processamento': 5.824188232421875, 'ta...",500,pt-BR,medio,ok,NaN,0.115385,0.034335,0.076923,1.192576e-06,0.816137,0.647328,False
3,Be prepared for your decision being seen by so...,How to Tell People You're Keeping Your Maiden ...,"However, traditions are not set in stone. In s...","Hoje em dia, não é mais obrigatório a mulher m...",NaN,"{'tempo_processamento': 6.02925968170166, 'tam...",500,pt-BR,medio,ok,NaN,0.160214,0.048193,0.101469,1.070240e-04,0.822365,0.681220,False
4,Pick a child that is three to seven years old....,How to Ask Someone to Be Your Ring Bearer,A general rule for the ring bearer is a child ...,Para escolher a criança que vai levar as alian...,NaN,"{'tempo_processamento': 4.799262285232544, 'ta...",500,pt-BR,medio,ok,NaN,0.180288,0.089157,0.120192,5.672125e-05,0.838326,0.646150,False
